# GFPGAN Colab Demo (Fork)

This Colab notebook installs dependencies, clones this fork, downloads pretrained weights automatically, and runs the GFPGAN inference script on sample images.

In [ ]:
#@title Install dependencies
%%bash
set -euxo pipefail
python -V
pip install --upgrade --quiet pip setuptools wheel
# Ensure IPython deps are satisfied to avoid resolver warnings
pip install --quiet -U ipython jedi
# Install torch (GPU wheels if NVIDIA present)
if command -v nvidia-smi >/dev/null 2>&1; then
  echo 'Detected NVIDIA GPU; installing CUDA wheels'
  pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else
  echo 'No NVIDIA GPU; installing CPU wheels'
  pip install --quiet torch torchvision torchaudio
fi
# Core deps: Use BasicSR master to match modern torchvision API
pip install --quiet --no-cache-dir --upgrade "git+https://github.com/xinntao/BasicSR@master" facexlib realesrgan opencv-python tqdm numpy PyYAML


In [ ]:
#@title Clone this fork
%%bash
set -euxo pipefail
if [ ! -d GFPGAN ]; then
  git clone --depth 1 https://github.com/IAmJonoBo/GFPGAN.git
fi
cd GFPGAN
echo "Repo at: $(pwd)"
ls -la inputs || true


In [ ]:
#@title Run inference on sample images
%%bash
set -euxo pipefail
cd GFPGAN
python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.4 -s 2
echo "Results written to ./GFPGAN/results"


In [ ]:
# @title Display a few restored images
from IPython.display import display, Image
import glob
import os

base = "GFPGAN/results/restored_imgs"
imgs = sorted(glob.glob(os.path.join(base, "*")))[:4]
print(f"Displaying {len(imgs)} images from", base)
for p in imgs:
    display(Image(filename=p))

In [ ]:
# @title Upload your own images and run GFPGAN
# @markdown Upload one or more images; they will be processed with the settings below.
from google.colab import files
import os
import shutil
import subprocess
import sys

upload_dir = "GFPGAN/uploads"
os.makedirs(upload_dir, exist_ok=True)
uploaded = files.upload()
for name, data in uploaded.items():
    # Save uploaded file and move into repo uploads folder
    with open(name, "wb") as f:
        f.write(data)
    shutil.move(name, os.path.join(upload_dir, os.path.basename(name)))

# Inference parameters
version = "1.4"  # @param ['1', '1.2', '1.3', '1.4']
scale = 2  # @param {type: 'integer'}
only_center_face = False  # @param {type: 'boolean'}
aligned = False  # @param {type: 'boolean'}

print("Running inference on uploads...")
cmd = [sys.executable, "inference_gfpgan.py", "-i", "uploads", "-o", "results", "-v", str(version), "-s", str(scale)]
if only_center_face:
    cmd.append("--only_center_face")
if aligned:
    cmd.append("--aligned")
subprocess.run(cmd, cwd="GFPGAN", check=False)
print("Done. See GFPGAN/results")

Notes:
- GPU acceleration is optional; uncomment the CUDA wheel index in the install cell to use GPU on Colab.
- The inference script will automatically download the GFPGAN v1.4 weights if not present.
- For large batches, consider enabling the Real-ESRGAN background upsampler by keeping default settings (it is auto-disabled on CPU).